In [1]:
import pandas as pd
from dspy.datasets import DataLoader
file_path = "../.data/all_raretrible.txt"
data = open(file_path, "r", encoding="utf-8").read().split("\n")
data = [x.strip().split(",") for x in data if len(x) > 0]
data = [[x[0], ",".join(x[1:])] for x in data]

columns=["label", "text",]
# 转换 data 到 pandas DataFrame
df = pd.DataFrame(data, columns=columns)

# 创建 dspy.datasets 对象
dl = DataLoader()
# fields 选择指定列 input_keys
all_dataset = dl.from_pandas(
    df,
    fields=columns,
    input_keys=("text",),
)
from collections import defaultdict
import random


def stratified_sample(from_dataset, label_attr, *dataset_sizes):
    label_groups = defaultdict(list)
    for item in from_dataset:
        label = getattr(item, label_attr)
        label_groups[label].append(item)

    result_sets = [[] for _ in dataset_sizes]

    for label, items in label_groups.items():
        random.shuffle(items)
        label_sizes = [size // len(label_groups) for size in dataset_sizes]
        start = 0
        for i, size in enumerate(label_sizes):
            end = start + size
            result_sets[i].extend(items[start:end])
            start = end

    all_items = [item for items in label_groups.values() for item in items]
    for i, (result_set, target_size) in enumerate(zip(result_sets, dataset_sizes)):
        shortage = target_size - len(result_set)
        if shortage > 0:
            result_sets[i].extend(random.sample(all_items, shortage))

    return result_sets
train_set, test_set, sample_set = stratified_sample(all_dataset, "label", 20, 50, 100)
from collections import Counter

train_set, test_set, sample_set = stratified_sample(all_dataset, "label", 20, 50, 100)
for dataset in [train_set, test_set, sample_set]:
    label_counts = Counter(item.get("label") for item in dataset)
    print(label_counts)

Counter({'感激': 4, '中性': 3, '高兴': 3, '抱怨': 3, '生气': 3, '焦急': 2, '惊讶': 2})
Counter({'高兴': 8, '感激': 7, '中性': 7, '焦急': 7, '惊讶': 7, '抱怨': 7, '生气': 7})
Counter({'感激': 15, '生气': 15, '中性': 14, '焦急': 14, '惊讶': 14, '高兴': 14, '抱怨': 14})


In [2]:
test_set,_ = stratified_sample(all_dataset, "label", 50,1)
import importlib
import transfer_lang

importlib.reload(transfer_lang)
# 印尼、泰、马来、菲律宾、粤语、日语、韩语
# trans = transfer_lang.TransferLang.from_dataset(test_set,lang="马来语")
# trans.modelname = 'glm-4-air'
# ret = trans.do()
# print(ret.content)
# trans.update_set()

for item in test_set:
    print(item)
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set,random=True,modelname = 'gpt-4o-mini')
ret = ana.do()
print(ret)
ana.update_dataset()
ana.show_main_error()

Example({'label': '感激', 'text': '谢谢你,找到了哪些书,但不知道怎么收藏'}) (input_keys={'text'})
Example({'label': '感激', 'text': '好的,那谢谢你'}) (input_keys={'text'})
Example({'label': '感激', 'text': '给我素万那普机场还车定位,谢谢!'}) (input_keys={'text'})
Example({'label': '感激', 'text': '好的,谢谢小姐姐哦'}) (input_keys={'text'})
Example({'label': '感激', 'text': '分这么多种类呀,还以为就一种呢,那知道了,谢谢!'}) (input_keys={'text'})
Example({'label': '感激', 'text': '哦,这样啊,谢谢'}) (input_keys={'text'})
Example({'label': '感激', 'text': '好谢谢回答拜拜拜哈哈哈'}) (input_keys={'text'})
Example({'label': '中性', 'text': '需要重新开机一次吗'}) (input_keys={'text'})
Example({'label': '中性', 'text': '我已经还电池'}) (input_keys={'text'})
Example({'label': '中性', 'text': '是不是小学还没有用这个平台'}) (input_keys={'text'})
Example({'label': '中性', 'text': '是不是越久越好呀'}) (input_keys={'text'})
Example({'label': '中性', 'text': '忘记哈稀值是什么了'}) (input_keys={'text'})
Example({'label': '中性', 'text': '和在那里上车呢'}) (input_keys={'text'})
Example({'label': '中性', 'text': '我已经在你们旁边机子上借了一个,可以了'}) (input_keys={'text'})
Example({

In [3]:
ana.show_error()

0.36
Example({'label': '生气', 'text': '你们的客户经理骗了我一千块钱', 'pred_label': '抱怨', 'main': '负面', 'score': '0.95'}) (input_keys={'text'})
Example({'label': '生气', 'text': '不然客户要准备投诉了', 'pred_label': '焦急', 'main': '负面', 'score': '0.85'}) (input_keys={'text'})
Example({'label': '生气', 'text': '我出来向王负责人反应情况时,拍视频者出来破口大骂,我丈夫从楼上下来劝,还被她打了头,当时很多一教练、员工都在场', 'pred_label': '抱怨', 'main': '负面', 'score': '0.95'}) (input_keys={'text'})
Example({'label': '焦急', 'text': '我买过的除非个别网站,我现在买的这两个网站都不会这么久', 'pred_label': '抱怨', 'main': '负面', 'score': '0.80'}) (input_keys={'text'})
Example({'label': '生气', 'text': '为什么每次都是非要联系了客服才能到呢?不联系就死活不到?这是什么做法?', 'pred_label': '抱怨', 'main': '负面', 'score': '0.90'}) (input_keys={'text'})
Example({'label': '惊讶', 'text': '我昨天买的今天不是应该确认吗', 'pred_label': '抱怨', 'main': '负面', 'score': '0.85'}) (input_keys={'text'})
Example({'label': '惊讶', 'text': '早就还了,怎么扣那么多钱', 'pred_label': '抱怨', 'main': '负面', 'score': '0.95'}) (input_keys={'text'})
Example({'label': '高兴', 'text': '真棒?', 'pred_label': '中性',

In [4]:
test_set,_ = stratified_sample(all_dataset, "label", 50,1)
import importlib
import transfer_lang

importlib.reload(transfer_lang)
# 印尼、泰、马来、菲律宾、粤语、日语、韩语
trans = transfer_lang.TransferLang.from_dataset(test_set,lang="马来语")
trans.modelname = 'gpt-4o-mini'
ret = trans.do()
print(ret.content)
trans.update_set()

for item in test_set:
    print(item)
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set,random=True,modelname = 'gpt-4o-mini')
ret = ana.do()
print(ret)
ana.update_dataset()
ana.show_main_error()

把下面内容翻译成 马来语: 
1: 好的,明白了。谢谢!
2: 好的好的,谢谢亲/::*/::*/::*
3: 好的,谢谢你了[emoji046]
4: 帮助我吧,谢谢你
5: 好的,谢谢,以为没发
6: 收到啦谢谢
7: 嗯嗯,谢谢你了哈
8: 好的谢谢现在买还发货吗?
9: 太宁路
10: 昨天我已经联系过你们啦
11: 我妈刚给我发的零花钱就扣没了
12: 订单多少时间是在超期以内
13: 我已经反馈客服了
14: 人事工号没发提供
15: 你快点给我解决啊
16: 物流那么慢的吗?不是照片出关成功了吗?
17: 为什么这么久还没有退
18: 订好几天了,怎么还没发货?
19: 麻烦尽快处理哦,谢谢
20: 尽快,太久我就去微博发一下了
21: 尽量催一下吧,谢谢
22: 湖南能用???这个
23: 这种情况真的太夸张了
24: 我的账号怎么不在了
25: 你们有真的人工服务?
26: 应该没事了吧,好吓人
27: 真牛了
28: 我够买的物料,显示五号就停止接单了,这又收到货了,怎么解释
29: 祝生日快乐!
30: [emoji033]哈哈
31: 很高兴认识
32: 亲爱哒
33: 好啦么么哒
34: 我非常满意
35: 出来了哈哈哈哈
36: 那个地方根本点不出
37: 啥故障呀,不能直接教我处理,我己到你们那里再寄回给我这时间都过去半个月了
38: 按完还没反应
39: 交易单号提供给你们了,这事有这么费劲这么难么?
40: 毕业了怎么你们没有发毕业证[emoji036]
41: 手机系统更新后,分身打不开了
42: 会员群组被踢了不给拉进去了?
43: 这么坑。。
44: 干嘛还要录视频?录视频是干嘛?看看我这个是不是活的?
45: 如果你们这样不负责任,以后还有谁用你们平台呢?你们代发的时候也不自捡吗?代发的代理也不检查检查的
46: 本来就是你们收了质检费结果到货还是那么明显质量问题
47: 这他妈都几天了好不能用
48: 都跟你说了,两号陆续发货,我说我等着回老家,让你两号就给我发,你觉得这是无理取闹吗?
49: 你工号多少,我截图。如果没有归还我就投诉呢
50: 哈哈哈

以下是内容翻译成马来语：

1: Baiklah, faham. Terima kasih!
2: Baiklah, terima kasih sayang/::*/::

In [5]:
test_set,_ = stratified_sample(all_dataset, "label", 50,1)
import importlib
import transfer_lang

importlib.reload(transfer_lang)
# 马来语 印尼语 泰语 马来语 菲律宾语 粤语 日语 韩语
trans = transfer_lang.TransferLang.from_dataset(test_set,lang="马来语")
trans.modelname = 'gpt-4o-mini'
ret = trans.do()
print(ret.content)
trans.update_set()

for item in test_set:
    print(item)
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set,random=True,modelname = 'gpt-4o-mini')
ret = ana.do()
print(ret)
ana.update_dataset()
ana.show_main_error()

把下面内容翻译成 马来语: 
1: 谢谢了,好人啊
2: 谢谢啦亲,
3: 好的!谢谢亲
4: 好饿,谢谢
5: 没事了哈,谢谢
6: 好的,谢谢!可以了
7: 是吗?那就好,非常感谢!
8: 中国是和平的
9: 人来了没
10: 还款审核中啥意思
11: 那意思是我们的贷款最后是打到公司了么
12: 我明天有菜,请帮忙节前一份金针菇给配上,谢谢了
13: 你们都收到货很久了
14: 我这个点剩多少钱呢?
15: 我现在很紧急
16: 了解了,希望您们能尽快帮我发下货吧,谢谢
17: 麻烦多催两遍,谢谢!
18: 麻烦了,尽快,谢谢
19: 麻烦你们快点
20: 很着急,找了三次了,都没有处理好
21: 这么多天不给顾客处理,我天天被吵的烦死了,这边能早点解决吗
22: 什么鬼
23: 这个你们不管?
24: 吓死人啊
25: 比抢火车票还厉害
26: 你们扣钱这么快吗
27: 不是吧?有没有搞错
28: 今天一看吓死还在计费
29: 爱你哦~
30: 昂哈哈哈哈
31: 哈哈哈哈
32: 我爱你么么哒
33: 再次㊗️您新年快乐
34: [emoji047]真好
35: 好啦么么哒
36: 欠费了然后我现在缴费了为什么还是用不了
37: 很烦
38: 我自己诊断恢复上网根本不好使
39: 这个坏的
40: 退款的怎么一直没客服联系我
41: 净网行动如此不力?
42: 这个又那么贵,少一盒就是几百块钱。
43: 这不是骗钱呢
44: 为啥就得你们联系供应商?我不能联系吗?这么久不给我发货?这是啥原因,我要联系供应商投诉
45: 真他妈的火大
46: 我他妈的哪里丢失了
47: 快递的失误,商家也不讲诚信,只顾自己,我是拉夏贝尔的粉丝,以后真的不敢再来了
48: 这就是恶意行为
49: 你除了逼逼这句其他不会
50: 才发现没归还到

以下是将内容翻译成马来语的结果：

1: Terima kasih, orang baik.
2: Terima kasih, sayang.
3: Baiklah! Terima kasih, sayang.
4: Sangat lapar, terima kasih.
5: Tak ada apa-apa, terima kasih.
6: Baiklah, terima kasih! Cukup.
7: Betulkah? It

In [6]:
test_set,_ = stratified_sample(all_dataset, "label", 50,1)
import importlib
import transfer_lang

importlib.reload(transfer_lang)
# 马来语 印尼语 泰语 马来语 菲律宾语 粤语 日语 韩语
trans = transfer_lang.TransferLang.from_dataset(test_set,lang="泰语")
trans.modelname = 'gpt-4o-mini'
ret = trans.do()
print(ret.content)
trans.update_set()

for item in test_set:
    print(item)
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set,random=True,modelname = 'gpt-4o-mini')
ret = ana.do()
print(ret)
ana.update_dataset()
ana.show_main_error()

把下面内容翻译成 泰语: 
1: 谢谢,辛苦了,过年好
2: 好的好的,谢谢,麻烦了
3: 啊谢谢我试试
4: 你好,我的问题目前解决了,谢谢
5: 好的,明白啦!谢谢
6: 好吧谢谢奥
7: 不用了,谢谢我已经找客服解决了,谢谢。
8: 缓慢摇头那
9: 总于好了
10: 请问目前苏州市区哪里还能配送,谢谢
11: 录完进货单没有库存
12: 你们有账单,我这也没有到账信息,以前的每一笔账都会有信息的,如果真的到账了,我也不会来找你问的
13: 本人亲,拍错了
14: 按摩机突然不运作啦
15: 好,好好尽快哦!
16: 太太太太慢了
17: 麻烦给我快点哦好的谢谢!
18: 我这怎么还没有啊
19: 麻烦您帮忙查一下,尽快安排发货,谢谢了
20: 嗯嗯,麻烦您操心催下呗,我这边只能看第一页,太耽误事呢
21: 等的人心累
22: 真牛了
23: 我的账号怎么不在了
24: 玩吧为什么最近总是闪退?
25: 充电宝只充了不到两个小时就没电了?
26: 妈呀。。。
27: 你们有真的人工服务?
28: 今天一看吓死还在计费
29: 哈哈哈哈??
30: 书太棒了!
31: 哈哈哈哈??
32: [emoji047]真好
33: Baiklah, muah muah
34: 哈哈哈哈。
35: 今天很开心啊
36: 我这中午时间全浪费在这
37: 可是我的专属客服不理我
38: 柜机都不在,重启个屁啊
39: 明明是你们的定位不准确
40: 我根本就还不进去
41: 钱不是白掏了
42: 康迪克水杯又漏水了
43: 眼睛瞎了还是让你妈尿尿呲了
44: 不要给我放屁,我又不是没遇到过。人家亲口承认你们游戏的
45: 麻烦您叫他们撤走,真气人
46: 老子不要处理了。你们拿着老子的钱去吃药
47: 真心感觉你们这个是一步步骗人的,兑你们这点钱真的老火,
48: 你们好坑哦!我告诉你们,我不会再续订你们的产品的了!绝对不会
49: 这样坑用户的钱
50: 感觉智商被这个平台侮辱了……

以下是将您提供的内容翻译成泰语的结果：

1: ขอบคุณค่ะ ทุ่มเทมากเลย สวัสดีปีใหม่
2: โอเค โอเค ขอบคุณค่ะ ขอโทษที่รบกวน
3: อ๋อ ขอบคุณค่ะ ฉันจะลองดู
4: สวัสดีค่ะ ปัญหาของ

In [7]:
test_set,_ = stratified_sample(all_dataset, "label", 50,1)
import importlib
import transfer_lang

importlib.reload(transfer_lang)
# 印尼语 泰语 马来语 菲律宾语 粤语 日语 韩语
trans = transfer_lang.TransferLang.from_dataset(test_set,lang="菲律宾语")
trans.modelname = 'gpt-4o-mini'
ret = trans.do()
print(ret.content)
trans.update_set()

for item in test_set:
    print(item)
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set,random=True,modelname = 'gpt-4o-mini')
ret = ana.do()
print(ret)
ana.update_dataset()
ana.show_main_error()

把下面内容翻译成 菲律宾语: 
1: 好了没有了多谢辛苦了拜拜
2: 嗯嗯,好的,谢谢你哦
3: 谢谢啦……
4: 不行,谢谢
5: 谢谢搞好了
6: 嗯,谢谢你啊
7: 谢谢亲,,,
8: 昨天说改成功了
9: 四块钱一个小时
10: 好的?谢谢,还有汇拓客平台商户激活码失效了没激活,是需要重新进件吗
11: 我没还错
12: 还是可以春节后报
13: 这些着有货啊
14: 这个我清楚
15: 谢谢,麻烦帮助催一下
16: 我求求你们快一点点啊!!!
17: Faham, harap kalian dapat hantar barang secepat mungkin, terima kasih.
18: 麻烦效率快点
19: 顾客都快要放假回老家了
20: 麻烦快点!谢谢
21: 好慢慢啊
22: 怎么这样呀
23: 这个是最后一个退货?
24: 真牛了
25: 这是真的不回我了?
26: 那也太恐怖啦!
27: 早就还了,怎么扣那么多钱
28: 还带插队的?
29: 哈哈哈你好巧巧
30: 很高兴认识
31: 哇太好了
32: 祝你小年愉快!
33: 啊哈哈哈哈哈哈
34: 爱你呦
35: 爱你呦
36: 上次快递送过来都烂了
37: 你们一直没给我回复呢
38: 真不想要你们产品太麻烦了
39: 不如快递柜快捷便利,人工就是慢
40: 我想问下,助长素小孩吃了怎么拉肚子,呕吐?
41: 同一时间的魔道进图慢半拍
42: 反映了很长时间一直解决不了
43: 你到底有么听懂我说什么吗?
44: 然后过两天又找你这敷衍人的客服?重新再说一遍?
45: 我现在是想投诉你们在线客服
46: 我出来向王负责人反应情况时,拍视频者出来破口大骂,我丈夫从楼上下来劝,还被她打了头,当时很多一教练、员工都在场
47: 不提醒谁他妈懂?
48: 大过年的,玩到你这游戏算我倒了八辈子大霉
49: 但搞得我心里很不爽。
50: 我先寄还是

以下是您提供内容的菲律宾语翻译：

1: Ayos na, wala na. Salamat sa hirap mo, paalam.
2: Hmm, okay, salamat sa iyo.
3: Salamat…
4: Hindi pwede, salamat.
5: Salamat, ayos na.
6: Hmm, salamat sa

In [8]:
test_set,_ = stratified_sample(all_dataset, "label", 50,1)
import importlib
import transfer_lang

importlib.reload(transfer_lang)
# 印尼语 泰语 马来语 菲律宾语 粤语 日语 韩语
trans = transfer_lang.TransferLang.from_dataset(test_set,lang="粤语")
trans.modelname = 'gpt-4o-mini'
ret = trans.do()
print(ret.content)
trans.update_set()

for item in test_set:
    print(item)
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set,random=True,modelname = 'gpt-4o-mini')
ret = ana.do()
print(ret)
ana.update_dataset()
ana.show_main_error()

把下面内容翻译成 粤语: 
1: 谢谢?。辛苦你了!
2: 非常感谢你的帮助,谢谢
3: 好的,谢谢[emoji003]
4: 我的问题解决了,谢谢你?
5: 哦,明白了,谢谢,打扰你了
6: 可以了!非常感谢!
7: 哦,谢谢哈!
8: 成绩有错
9: 但是这边没提示
10: 微信小程序我截下图
11: 哪些属于偏远
12: 那可能是没看到的
13: 昨天还能取消的啊
14: 我真的没骂人。
15: 那怎么还在审核中
16: 改一下地址吧!客人催得好紧,麻烦快点
17: 能帮我加急处理吗,项目上的工人情绪都很大,谢谢
18: เครื่องนวด突然ไม่ทำงานแล้ว
19: 能快点不,亲
20: 优先发,帮我催快点,谢谢了
21: 为什么身份认证需要这么久?
22: 这种情况真的太夸张了
23: 就是坑坑洼洼的,你可能看到??
24: 还要收费???
25: 不是吧?有没有搞错
26: 你们扣钱这么快吗
27: 我昨天买的今天不是应该确认吗
28: 不交诚意金聊不了嘛不交诚意金聊不了嘛
29: 好可爱
30: 哈哈好像也太搞笑了
31: 真棒?
32: 么么哒?
33: 非常好
34: 太期待了
35: 哈哈哈哈
36: 开个网页都费劲
37: 时间都是一样的怎么一个进度快一个进度慢哦
38: 一到退款就这么麻烦
39: 说的给发,一直没看到
40: 你们电话太难打了
41: 这个老是断网啊
42: 我对这个游戏很失望
43: 垃圾垃圾垃圾
44: 很崩溃,如果没有你们的充电桩,不能在系统中及时更新一下吗?以及连投诉反馈该归还点的位置都没有,让人白跑
45: 他骂我,我回击他,反过来禁我言!
46: 你死全家
47: 越来越**
48: 一个月都没用上呢?这不是骗人吗?
49: 我气真的不打一处了
50: 哪里可以举报,投诉

以下是你提供的内容翻译成粤语：

1: 多谢你啊！辛苦你啦！
2: 非常感谢你嘅帮助，多谢。
3: 好嘅，多谢[emoji003]。
4: 我嘅问题解决咗，多谢你啊？
5: 哦，明白啦，多谢，打扰你啦。
6: 可以啦！非常感谢！
7: 哦，多谢哈！
8: 成绩有错。
9: 但系呢边冇提示。
10: 微信小程序我截咗图。
11: 边啲属于偏远。
12: 咁可能系冇睇到嘅。
13: 昨日仲可以取消嘅啊。
14: 我真系冇骂人。
15: 

In [10]:
test_set,_ = stratified_sample(all_dataset, "label", 50,1)
import importlib
import transfer_lang

importlib.reload(transfer_lang)
# 印尼语 泰语 马来语 菲律宾语 粤语 日语 韩语
trans = transfer_lang.TransferLang.from_dataset(test_set,lang="日语")
trans.modelname = 'gpt-4o-mini'
ret = trans.do()
print(ret.content)
trans.update_set()

for item in test_set:
    print(item)
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set,random=True,modelname = 'gpt-4o-mini')
ret = ana.do()
print(ret)
ana.update_dataset()
ana.show_main_error()

Please provide the content you want to be translated into 日语: 
1: 好好谢谢亲爱的
2: 嗯嗯嗯嗯,好的谢谢麻烦你了。
3: 好哒好哒,辛苦啦。
4: [emoji048]谢谢,拜拜
5: 谢谢啦,知道啦
6: 明白,谢谢哈
7: 好的,谢谢,平台好了发个信息给我
8: 请您再看看照片行不行
9: 第二个订单,女鞋
10: 那为什么?
11: 导致一次货款都没有打过
12: 联系谁可以知道这个单据的情况
13: 我的意思,不用审了,我不用了,现在等没结果一个星期了!我到朋友跟借点就可以!咋们这个可以给我扯消订单吗?
14: 就是你们的白菜肯定不会喷甲醛吧
15: 三天了都没到账
16: 急死了快
17: 这个事情能尽快帮我一下吗
18: 还请快点,谢谢
19: 辛苦快点
20: 我前两天下的单怎么还没发货呢?
21: 我想问下我下单的红酒怎么还没发货了
22: 早就还了,怎么扣那么多钱
23: 玩吧为什么最近总是闪退?
24: 什么速度我靠
25: 我看下都吓我一跳
26: 这个是最后一个退货?
27: 今天一看吓死还在计费
28: 卧槽...他如果不还呢?
29: 昂哈哈哈哈
30: 好可爱
31: 啊哈哈哈哈哈哈
32: 哈哈哈哈对耶
33: 昂哈哈哈哈
34: 很高兴与您的对话哦
35: 搞定了哈哈哈哈
36: 换了新版本怎么还是打印不了
37: 那慢也不是这样慢呀,你们和他们合作你们不能管控的吗?
38: 第一次这么反感
39: 半天不回问题
40: 你的意思我自认倒霉?
41: 老是不回复
42: 都用着好麻烦
43: 就这么糊口答应
44: 瞎在这里乱收钱
45: 搞什么东西呀。态度差的要死
46: 你妈死了没
47: 算了吧,你继续装死吧,再见。
48: 还请快点,谢谢  
49: 我该去哪里投诉呢?
50: 谢谢您放心了

 Translation text:
Here is the translation of the provided content into Japanese:

1: いいね、ありがとう、親愛なる。
2: うんうん、わかった、ありがとう、面倒をかけて。
3: はいはい、ありがとう、お疲れ様。
4: [emoji048]ありがとう、バイバイ。
5: ありがとう、わか

In [11]:
test_set,_ = stratified_sample(all_dataset, "label", 50,1)
import importlib
import transfer_lang

importlib.reload(transfer_lang)
# 印尼语 泰语 马来语 菲律宾语 粤语 日语 韩语
trans = transfer_lang.TransferLang.from_dataset(test_set,lang="韩语")
trans.modelname = 'gpt-4o-mini'
ret = trans.do()
print(ret.content)
trans.update_set()

for item in test_set:
    print(item)
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set,random=True,modelname = 'gpt-4o-mini')
ret = ana.do()
print(ret)
ana.update_dataset()
ana.show_main_error()

Please provide the content you want to be translated into 韩语: 
1: 好的,谢谢亲,我考虑一下。
2: 好谢谢了没事了
3: 行吧,谢了,麻烦你们了
4: ?谢谢呀
5: 行,谢谢,没其他问题了
6: 我明白了,谢谢哈
7: 不是,我找到了,谢谢哈
8: 要不我发红包给你,你直接用通道给我充钱
9: 明天白天我问问他
10: 你们官方商城都没货,那我哪里能买到?
11: 她的账户怎么锁定?
12: 手表有没有哪些地方需要注意一下
13: 到底做好没啊
14: 是不是不出了啊
15: 好的,谢谢,请快点转账
16: 昨日仲可以取消嘅啊。
17: 没有,就希望这个问题能帮忙尽快的处理。
18: 我求求你们快一点点啊!!!
19: 麻烦快点受理
20: 好的,麻烦您加急了
21: 我不想等太久了我下午还有事要做
22: 我的账号怎么不在了
23: 半个小时还没变?
24: 不是吧?有没有搞错
25: 然后一直扣费?
26: 你还让我给你好评吗?
27: 你们有真的人工服务?
28: 你们发货那么超前
29: 哈哈哈好吧
30: 哈哈哈好的
31: 啊哈哈哈哈哈哈
32: 欢乐
33: 很高兴认识你
34: 快乐快乐
35: Hahaha, ang taas ng tsansa mo.
36: 今天体验还是不好,没有太多互动,一直在纠音,学生都是一脸懵
37: 提个现真多限制
38: 质量不好不是吗
39: 对你们得物流服务我很质疑
40: 脸上有妊辰斑用什么可以除去  
41: 丢到反馈都很久了
42: 客服也解决不了啊
43: 草泥马的作弊
44: 你们都在踢皮球
45: 这个票,被废,非常恶劣。我相信你们供应也不可能去随便废,希望能有个合理的解释。
46: 那你们给额度干嘛,闲的没事干逗我玩啊
47: 死你马
48: 我不想讲了,上次的体验非常糟糕
49: Jika anda tidak bertanggungjawab begini, siapa lagi yang akan menggunakan platform anda? Apabila anda menghantar, adakah anda tidak memeriksa? Ejen penghantaran juga tidak periks